In [1]:
import pandas as pd

In [8]:
url = "./make_file/(위경도)_서울시_도로_정리(기점_종점).csv"

In [9]:
data = pd.read_csv(url)

In [10]:
data.columns

Index(['Unnamed: 0', '시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이'], dtype='object')

In [11]:
try :
    data = data.drop(columns='Unnamed: 0')
except KeyError as e :
    print(e)

In [12]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이'], dtype='object')

In [13]:
start_latitude = data['시점_위도'].to_list()
start_longitude = data['시점_경도'].to_list()

end_latitude = data['종점_위도'].to_list()
end_longitude = data['종점_경도'].to_list()

start_altitude = []
end_altitude = []

In [14]:
print(len(start_latitude))
print(len(start_longitude))
print(len(end_latitude))
print(len(end_longitude))

144
144
144
144


## GOOGLE API

In [15]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()


사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
Name: count, dtype: int64

In [16]:
GOOGLE_API_KEY = open_api_key_file[open_api_key_file['사이트'] == 'GOOGLE_MAP'].values[0][1]  

In [17]:
import requests
import time
import pandas as pd 

# Google Elevation API 호출 함수
def google_api(latitudes, longitudes, mode):
    global start_altitude, end_altitude
    
    url = f"https://maps.googleapis.com/maps/api/elevation/json?key={GOOGLE_API_KEY}"

    locations = list(zip(latitudes, longitudes))  # (위도, 경도) 쌍 리스트
    
    for i in range(0, len(locations), 512):  # 최대 512개씩 요청
        batch_locations = locations[i:i+512]
        locations_param = '|'.join([f"{lat},{lon}" for lat, lon in batch_locations])
        full_url = f"{url}&locations={locations_param}"
        
        response = requests.get(full_url)
            
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'OK':
                elevations = [result['elevation'] for result in data['results']]   
            
                if mode == 'start':
                    start_altitude.extend(elevations)
                elif mode == 'end':
                    end_altitude.extend(elevations)
            else:
                print("Error in response:", data['status'])
        else:
            print("API 호출 실패:", response.status_code)

        time.sleep(0.1)  # API 호출 간 지연 시간

In [18]:
print(f'전) 기점 고도 길이 : {len(start_altitude)}') 

print(f'전) 종점 고도 길이 : {len(end_altitude)}') 

전) 기점 고도 길이 : 0
전) 종점 고도 길이 : 0


In [20]:
# API 호출 (start)
print(f'기점의 고도값 불러오기')
print(f'시작')
google_api(start_latitude, start_longitude, 'start') 
print(f'종료')

기점의 고도값 불러오기
시작
종료


In [19]:
# API 호출 (end)
print(f'종점의 고도값 불러오기')
print(f'시작')
google_api(end_latitude, end_longitude, 'end')
print(f'종료')

종점의 고도값 불러오기
시작
종료


In [21]:
print(f'후) 기점 고도 길이 : {len(start_altitude)}') 
print(f'후) 종점 고도 길이 : {len(end_altitude)}') 

if (len(start_altitude) == len(end_altitude)) and (len(start_altitude) > 0) :
    print(f'결과의 길이가 같습니다.')
    print(f'성공적으로 처리되었습니다.')
else :
    print(f'데이터 처리에 실패했습니다...')

후) 기점 고도 길이 : 144
후) 종점 고도 길이 : 144
결과의 길이가 같습니다.
성공적으로 처리되었습니다.


In [22]:
altitude_difference = []

for i in range(len(start_altitude)) :
    result = abs(start_altitude[i] - end_altitude[i]) 
    altitude_difference.append(result)

print(len(altitude_difference))

144


In [23]:
# 결과 데이터 추가 및 저장
data['시작점_고도'] = start_altitude
data['종료점_고도'] = end_altitude
data['고도의_차이'] = altitude_difference

In [28]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도', '종료점_고도', '고도의_차이'], dtype='object')

In [29]:
start_lat = data['시점_위도'].to_list()
start_lon = data['시점_경도'].to_list()
end_lat = data['종점_위도'].to_list()
end_lon = data['종점_경도'].to_list()
different_alt = data['고도의_차이'].to_list()

In [30]:
print(f'start_lat 길이 : {len(start_lat)}')
print(f'start_lon 길이 : {len(start_lon)}')
print(f'end_lat 길이 : {len(end_lat)}')
print(f'end_lon 길이 : {len(end_lon)}')
print(f'different_alt 길이 : {len(different_alt)}')

start_lat 길이 : 144
start_lon 길이 : 144
end_lat 길이 : 144
end_lon 길이 : 144
different_alt 길이 : 144


In [31]:
import math

# 지구 반지름 (미터)
R = 6371000   

In [32]:
# 결과 저장 리스트
road_slope = []

In [33]:
for i in range(len(start_lat)):
    # 위경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(math.radians, [start_lat[i], start_lon[i], end_lat[i], end_lon[i]])

    # 위도, 경도의 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine 공식으로 수평 거리(d) 계산
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c  # 두 지점 간 수평 거리 (m)

    # 고도 차이 (h)
    h = different_alt[i]  

    # 경사각(θ) 계산 (라디안 → 도 변환)
    theta = math.degrees(math.atan2(h, d))

    # 결과 저장
    road_slope.append(theta)

    if(i < 5) :
        # 디버깅 출력
        print(f"Point {i + 1}: d = {d:.4f} m, h = {h:.4f} m, θ = {theta:.4f}°")


Point 1: d = 153.0020 m, h = 31.7527 m, θ = 11.7242°
Point 2: d = 434.7030 m, h = 38.3723 m, θ = 5.0446°
Point 3: d = 289.7552 m, h = 39.9972 m, θ = 7.8593°
Point 4: d = 198.7964 m, h = 35.2075 m, θ = 10.0431°
Point 5: d = 307.6295 m, h = 6.5866 m, θ = 1.2266°


In [34]:
print(f'변환한 road_slope의 길이 : {len(road_slope)}')

변환한 road_slope의 길이 : 144


In [35]:
new_add_df = pd.DataFrame({
    '경사각' : road_slope
})

In [36]:
data['경사각'] = new_add_df

In [37]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도', '종료점_고도', '고도의_차이',
       '경사각'],
      dtype='object')

In [38]:
data['경사각']

0      11.724227
1       5.044559
2       7.859327
3      10.043141
4       1.226563
         ...    
139     1.485426
140     1.410358
141     0.365226
142     2.663688
143     3.676672
Name: 경사각, Length: 144, dtype: float64

In [41]:
try :
    save_data_url = './make_file/열선_위경도값(중앙값_고도_경사각_포함).csv'
    data.to_csv(save_data_url, encoding='euc-kr')
    print(f"{save_data_url} 저장 완료")
except :
    print("err") 

./make_file/열선_위경도값(중앙값_고도_경사각_포함).csv 저장 완료
